In [125]:
%matplotlib qt

from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../data/qbo_data.csv",index_col="run_id")
df = df.sort_values("qbo_period_std")
df

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,cw,Bt,qbo_periods,qbo_period_std,qbo_period_fft,qbo_count,qbo_rmse_ampltiude,qbo_amplitude_mean,qbo_amplitude_std
run_id,,,,,,,,,
78,63.111004,0.002280,11.466667,3.984022e-15,11.466667,19,12.475006,17.731903,0.000000
1,44.617037,0.005099,24.116463,8.522248e-01,24.000000,9,42.536683,39.017867,1.996191
64,49.812074,0.005277,23.683767,8.823445e-01,24.000000,9,46.075949,42.475577,1.897165
60,21.845038,0.006956,19.117979,1.263149e+00,18.461538,12,34.874135,35.291554,1.546429
15,33.698484,0.005147,24.227832,1.284637e+00,24.000000,9,40.913906,38.525333,2.027852
...,...,...,...,...,...,...,...,...,...
35,8.580488,0.005694,18.281435,1.201578e+01,26.666667,12,12.293360,11.247630,4.733746
52,17.041039,0.001164,28.288639,1.384765e+01,12.000000,8,10.699096,11.159743,3.234233
44,5.930840,0.004727,27.009087,1.428539e+01,12.000000,8,10.644768,11.106186,1.924817


## Constants

In [126]:
OBS_PERIOD_MEAN = 27.915249015787193
OBS_PERIOD_STD = 4.538404428603899

OBS_AMP_MEAN = 26.587500000000002
OBS_AMP_STD =2.953799762291663

CURRENT_CW = 35
CURRENT_BT = 0.0043

In [127]:
import numpy as np
X = df[["Bt","cw"]].to_numpy()
y_point_error = df[["qbo_period_std","qbo_amplitude_std"]].to_numpy()
y = df[["qbo_periods","qbo_amplitude_mean"]].to_numpy()

mean_y = np.mean(y,axis=0)
std_y = np.std(y,axis=0)
y_norm = (y - mean_y)/std_y
y_point_error = y_point_error/std_y

In [158]:
y_point_error

array([[1.02774244e-15, 0.00000000e+00],
       [2.19845045e-01, 1.88414334e-01],
       [2.27614906e-01, 1.79067564e-01],
       [3.25849513e-01, 1.45962694e-01],
       [3.31392590e-01, 1.91402801e-01],
       [3.51033182e-01, 3.16237812e-01],
       [3.83423465e-01, 2.18072117e-01],
       [3.85627921e-01, 1.53017733e-01],
       [3.92895762e-01, 2.62173246e-01],
       [3.93179416e-01, 1.43690465e-01],
       [4.17391819e-01, 2.08905819e-01],
       [4.19144664e-01, 1.43515072e-01],
       [4.82716322e-01, 1.81856506e-01],
       [5.29789952e-01, 1.73120596e-01],
       [5.43330618e-01, 1.72986820e-01],
       [5.44813120e-01, 1.99120860e-01],
       [5.49191428e-01, 2.04555209e-01],
       [5.62558066e-01, 1.93461153e-01],
       [5.75387815e-01, 3.32316199e-01],
       [5.99663842e-01, 2.34307981e-01],
       [6.01757204e-01, 1.95536479e-01],
       [6.02043328e-01, 3.46357204e-01],
       [6.06576649e-01, 1.13505111e-01],
       [6.08729459e-01, 2.64889063e-01],
       [6.138537

In [128]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)




In [157]:
from sklearn.gaussian_process import GaussianProcessRegressor

gpr_period = GaussianProcessRegressor(alpha=(y_point_error[:,0])**2)
gpr_amplitude = GaussianProcessRegressor(alpha=(y_point_error[:,1])**2)
gpr_period.fit(X_scaled,y_norm[:,0])
gpr_amplitude.fit(X_scaled,y_norm[:,1])


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
predval,predstd = gpr_period.predict(scaler.transform([[0.0043,35]]),return_std=True)
predval = predval*std_y[0] + mean_y[0]
predstd = predstd*std_y[0]

In [ ]:
predval,predstd

(array([25.98154407]), array([0.70357845]))

### 3D Plots?

In [ ]:
CW_POINTS = 1000
BT_POINTS = 1000
cw = np.linspace(5,70,CW_POINTS)
Bt = np.linspace(0.001,0.007,BT_POINTS)
X_Bt,Y_cw = np.meshgrid(Bt,cw)
x_samples = np.vstack((X_Bt.ravel(),Y_cw.ravel())).T
x_samples = scaler.transform(x_samples)


In [ ]:
period,period_std = gpr_period.predict(x_samples,return_std=True)
period = period*std_y[0] + mean_y[0]
period = period.reshape((CW_POINTS,BT_POINTS))
period_std = (period_std*std_y[0]).reshape((CW_POINTS,BT_POINTS))


amplitude,amplitude_std = gpr_amplitude.predict(x_samples,return_std=True)
amplitude = amplitude*std_y[1] + mean_y[1]
amplitude = amplitude.reshape((CW_POINTS,BT_POINTS))
amplitude_std = (amplitude_std*std_y[1]).reshape((CW_POINTS,BT_POINTS))

In [ ]:
implausability_period = np.abs(period - OBS_PERIOD_MEAN)/np.sqrt(OBS_PERIOD_STD**2 + period_std**2)
implausability_amplitude = np.abs(amplitude - OBS_AMP_MEAN)/np.sqrt(OBS_AMP_STD**2 + amplitude_std**2)
implausability = np.sqrt(implausability_amplitude**2 + implausability_period**2)

In [ ]:
imp_cutoff = implausability < np.percentile(implausability,20)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,6))
CS = ax1.contour(X_Bt,Y_cw,amplitude,levels=25)
ax1.scatter(X[:,0],X[:,1],)
ax1.scatter(CURRENT_BT,CURRENT_CW)
ax1.clabel(CS, CS.levels, inline=True,  fontsize=10)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")

CS2 = ax2.contour(X_Bt,Y_cw,period,levels=25)
ax2.scatter(X[:,0],X[:,1],)
ax2.scatter(CURRENT_BT,CURRENT_CW)
ax2.clabel(CS2, CS2.levels, inline=True,  fontsize=10)
ax2.set_ylabel("Cw")
ax2.set_xlabel("Bt")

ax3.contour(X_Bt,Y_cw,implausability,levels=25)
ax3.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn',levels=1)
ax3.scatter(X[:,0],X[:,1])
ax3.scatter(CURRENT_BT,CURRENT_CW)
ax3.set_ylabel("Cw")
ax3.set_xlabel("Bt")

Text(0.5, 0, 'Bt')

In [ ]:
from mpl_toolkits.mplot3d import axes3d
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,implausability,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("Implausability")

Text(0.5, 0, 'Implausability')

## Calculate Decision Boundary + Sampling

In [146]:
def get_new_samples(new_sample_space,X,Y,n_samples=100):
    """
    MCMC sampling.
    Probably not very
    Dimensions of new_sample_space,X,Y, must match
    """
    ## Calculating bounding box of new_sample_space.
    y_where,x_where = np.where(new_sample_space)
    x_min,x_max,y_min,y_max = X[0,np.min(x_where)],X[0,np.max(x_where)],Y[np.min(y_where),0],Y[np.max(y_where),0]
    new_samples = []
    while len(new_samples) < n_samples:
        # Sample random point in original sample space 
        x,y = np.random.uniform(low=(x_min,y_min),high=(x_max,y_max))
        col = np.argmin(np.abs(X-x),axis=1)[1]
        row = np.argmin(np.abs(Y-y),axis=0)[0]
        if new_sample_space[row,col]:
            new_samples.append((x,y))
        # Check sample point meets plausability criterion
    return np.array(new_samples)


In [147]:
new_samples = get_new_samples(imp_cutoff,X_Bt,Y_cw)

### LHS

In [155]:
from pyDOE import lhs

def get_new_samples_lhs(new_sample_space,X,Y,n_samples=100,max_iter=1000):
    """
    MCMC sampling.
    Probably not very
    Dimensions of new_sample_space,X,Y, must match
    """
    ## Calculating bounding box of new_sample_space.
    y_where,x_where = np.where(new_sample_space)
    x_min,x_max,y_min,y_max = X[0,np.min(x_where)],X[0,np.max(x_where)],Y[np.min(y_where),0],Y[np.max(y_where),0]
    
    rescale = np.array([[x_max -x_min,0],[0,y_max-y_min],[x_min,y_min]])
    for i in range(1,max_iter):
        # Sample random point in original sample space 
        samples = lhs(2,samples=n_samples*i*5,criterion='m')
        samples = np.column_stack((samples,np.ones(n_samples*i*5))) @ rescale
        print(samples[:,1].min())
        rows,cols = (np.argmin(np.abs(Y[:,0,None] - samples[:,1][None,:]),axis=0),
                                  np.argmin(np.abs(X[0,:,None] - samples[:,0][None,:]),axis=0))
        valid = new_sample_space[cols,rows]
        if np.sum(valid) >= n_samples:
            ys,xs = Y[cols[valid],0],X[0,rows[valid]]
            return np.column_stack((xs,ys))
    raise RuntimeError(f"Unable to draw {n_samples} samples in {max_iter} iterations from the sample space")


In [156]:
lhs_samples = get_new_samples_lhs(imp_cutoff,X_Bt,Y_cw)

13.165514234716062


In [154]:
fig,(ax1) = plt.subplots(figsize=(8,8))


ax1.contour(X_Bt,Y_cw,implausability,levels=25)
ax1.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn',levels=1)
#ax1.scatter(CURRENT_BT,CURRENT_CW)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")
ax1.scatter(new_samples[:,0],new_samples[:,1],c='r',label="MCMC")
ax1.scatter(lhs_samples[:,0],lhs_samples[:,1],c='g',label="Latin Hypercube")
ax1.legend()